# Agfa DR600/DXD600 - Log Parser

This notebook takes the output of the EXI Log export from an Agfa DR600 X-ray system and produces a .csv file that can be appended to the Reject Analysis and Dose Metric Dashboard database.

Let's start by importing the required libraries needed to make the parser work.

In [1]:
import pandas as pd
import os

from ipywidgets import *
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

The template for the database used in the Reject Analysis and Dose Metric Dashboard is as follows:

In [2]:
cols_list_final = ['Asset Number','Manufacturer','Model','Image Date','Image Time','Body Part','View','Exposure Index','KAP (uGy.m2)','kVp','Exposure (mAs)','Exposure time (ms)','Image Status','Reject Reason']

df_template = pd.DataFrame(columns=[*cols_list_final])
df_template

Empty DataFrame
Columns: [Asset Number, Manufacturer, Model, Image Date, Image Time, Body Part, View, Exposure Index, KAP (uGy.m2), kVp, Exposure (mAs), Exposure time (ms), Image Status, Reject Reason]
Index: []

## Selecting file from file browser

To clean up an export, you'll need to select the exported files from the Philips QA Tool using the "File Select" button below.
<br>
You will also be asked for your asset number so that we can group systems in the Dashboard by facility.

In [7]:
# Select input log from file location

def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename() # List of selected files will be set button's file attribute.
    print(b.files) # Print the list of files selected.

fileselect = Button(description="File select")
fileselect.on_click(select_files)

display(fileselect)

C:/Users/bernardm/GitHub/JupyterNotebooks/rejectAnalysis/inputdata/Agfa DR600 example log.xlsx


## Adding Asset Number

In the next section, the file that you've selected will be printed out and you'll be asked to input the Asset Number of the system. Type the Asset Number in the box provided and then press "ENTER".

In [ ]:
files = fileselect.files
print("The file you've selected is: ", files)
print()
print("What is the asset number of the system?")

AssetNumber = input()


The file you've selected is:  C:/Users/bernardm/GitHub/JupyterNotebooks/rejectAnalysis/inputdata/Agfa DR600 example log.xlsx

What is the asset number of the system?


## Clean up dataframe

Let's do some clean-up on the .csv file:
- Define the separator. By default, this will be ";". However, this can be set to something different during export (| or , or -). Change the code below if your separator is something other than ";".

In [ ]:
df = pd.read_excel(files)
df

## Adding unique identifiers

If the .csv file has been read in correctly above, you should see a table with all the values from the log. 
<br>
Let's add a few identifiers for filtering purposes:
- Asset Number
- Manufacturer
- Model

The following columns are not present in the Agfa Log and have been set to "NULL" values:
- Exposure Index
- kVp
- Exposure (mAs)
- Exposure time (ms)
- KAP
- Image Time


In [23]:
cols_list = ['Asset Number','Manufacturer','Model']
df = df.reindex(columns=[*cols_list,*df.columns.tolist()])

df['Asset Number'] = AssetNumber
df['Manufacturer'] = "Agfa"
df['Model'] = "DR600"
df['Exposure Index'] = None
df['kVp'] = None
df['Exposure (mAs)'] = None
df['Exposure time (ms)'] = None
df['KAP (uGy.m2)'] = None
df['Image Time'] = None
df

Asset Number Manufacturer  Model Asset Number Manufacturer  Model  \
0     123456789         Agfa  DR600    123456789         Agfa  DR600   
1     123456789         Agfa  DR600    123456789         Agfa  DR600   
2     123456789         Agfa  DR600    123456789         Agfa  DR600   
3     123456789         Agfa  DR600    123456789         Agfa  DR600   
4     123456789         Agfa  DR600    123456789         Agfa  DR600   
5     123456789         Agfa  DR600    123456789         Agfa  DR600   
6     123456789         Agfa  DR600    123456789         Agfa  DR600   
7     123456789         Agfa  DR600    123456789         Agfa  DR600   
8     123456789         Agfa  DR600    123456789         Agfa  DR600   
9     123456789         Agfa  DR600    123456789         Agfa  DR600   
10    123456789         Agfa  DR600    123456789         Agfa  DR600   
11    123456789         Agfa  DR600    123456789         Agfa  DR600   
12    123456789         Agfa  DR600    123456789         Agfa  DR600   
13    123456789         Agfa  DR600    123456789         Agfa  DR600   
14    123456789         Agfa  DR600    123456789         Agfa  DR600   

          Hospital     Department AgeGroupName           ExamGroup  ...  \
0   Logan Hospital  Emergency Rm1          17+               Chest  ...   
1   Logan Hospital  Emergency Rm1          17+  Shoulder & Humerus  ...   
2   Logan Hospital  Emergency Rm1          17+               Chest  ...   
3   Logan Hospital  Emergency Rm1          17+               Chest  ...   
4   Logan Hospital  Emergency Rm1          17+               Chest  ...   
5   Logan Hospital  Emergency Rm1          17+               Chest  ...   
6   Logan Hospital  Emergency Rm1          17+               Chest  ...   
7   Logan Hospital  Emergency Rm1          17+               Chest  ...   
8   Logan Hospital  Emergency Rm1          17+               Chest  ...   
9   Logan Hospital  Emergency Rm1          17+               Chest  ...   
10  Logan Hospital  Emergency Rm1          17+               Chest  ...   
11  Logan Hospital  Emergency Rm1          17+               Chest  ...   
12  Logan Hospital  Emergency Rm1          17+               Chest  ...   
13  Logan Hospital  Emergency Rm1          17+               Chest  ...   
14  Logan Hospital  Emergency Rm1          17+               Chest  ...   

   Operator   Workstation RejectStatus       RejectReason  Exposure Index  \
0       AMP  AE_LOGEDXRM1     APPROVED                NaN            None   
1       LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
2       LDK  AE_LOGEDXRM1     REJECTED  Positioning error            None   
3       LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
4       LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
5       LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
6       AMP  AE_LOGEDXRM1     APPROVED                NaN            None   
7       AMP  AE_LOGEDXRM1     APPROVED                NaN            None   
8       AMP  AE_LOGEDXRM1     APPROVED                NaN            None   
9       LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
10      LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
11      LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
12      LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
13      LDK  AE_LOGEDXRM1     APPROVED                NaN            None   
14      LDK  AE_LOGEDXRM1     REJECTED     Anatomy cutoff            None   

     kVp Exposure (mAs) Exposure time (ms) KAP (uGy.m2) Image Time  
0   None           None               None         None       None  
1   None           None               None         None       None  
2   None           None               None         None       None  
3   None           None               None         None       None  
4   None           None               None         None       None  
5 

## Putting data into the template dataframe

Let's match the .csv columns into the template:
- rename the columns of the original .csv file to match the template
- remove any columns we don't need
- rearrange the columns to match the template

In [24]:
df = df.rename(columns={"BodyPart": "Body Part",
                       "ExposureType": "View",
                        "RejectStatus": "Image Status",
                        "RejectReason": "Reject Reason",
                        "AcquisitionDateTime": "Image Date",
                       })

df_out = df[[*cols_list_final]]
df_out

Asset Number Asset Number Manufacturer Manufacturer  Model  Model  \
0     123456789    123456789         Agfa         Agfa  DR600  DR600   
1     123456789    123456789         Agfa         Agfa  DR600  DR600   
2     123456789    123456789         Agfa         Agfa  DR600  DR600   
3     123456789    123456789         Agfa         Agfa  DR600  DR600   
4     123456789    123456789         Agfa         Agfa  DR600  DR600   
5     123456789    123456789         Agfa         Agfa  DR600  DR600   
6     123456789    123456789         Agfa         Agfa  DR600  DR600   
7     123456789    123456789         Agfa         Agfa  DR600  DR600   
8     123456789    123456789         Agfa         Agfa  DR600  DR600   
9     123456789    123456789         Agfa         Agfa  DR600  DR600   
10    123456789    123456789         Agfa         Agfa  DR600  DR600   
11    123456789    123456789         Agfa         Agfa  DR600  DR600   
12    123456789    123456789         Agfa         Agfa  DR600  DR600   
13    123456789    123456789         Agfa         Agfa  DR600  DR600   
14    123456789    123456789         Agfa         Agfa  DR600  DR600   

                 Image Date Image Time Body Part               View  \
0   2020-06-01T00:04:36.283       None     CHEST  No Grid ChestAP F   
1   2020-06-01T00:27:46.143       None  SHOULDER       Scapular Y F   
2    2020-06-01T00:45:56.69       None     CHEST     Grid ChestAP F   
3   2020-06-01T00:46:47.633       None     CHEST     Grid ChestAP F   
4   2020-06-01T00:58:08.673       None     CHEST         Chest PA W   
5   2020-06-01T00:58:36.323       None     CHEST        Chest Lat W   
6   2020-06-01T01:32:48.127       None     CHEST  No Grid ChestAP F   
7    2020-06-01T01:52:53.76       None     CHEST         Chest PA W   
8   2020-06-01T01:53:35.757       None     CHEST        Chest Lat W   
9   2020-06-01T02:38:03.167       None     CHEST         Chest PA W   
10  2020-06-01T02:38:22.043       None     CHEST        Chest Lat W   
11   2020-06-01T02:49:16.73       None     CHEST         Chest PA W   
12  2020-06-01T02:50:09.813       None     CHEST        Chest Lat W   
13  2020-06-01T03:11:50.677       None     CHEST         Chest PA W   
14  2020-06-01T04:20:32.633       None     CHEST  No Grid ChestAP F   

   Exposure Index KAP (uGy.m2)   kVp Exposure (mAs) Exposure time (ms)  \
0            None         None  None           None               None   
1            None         None  None           None               None   
2            None         None  None           None               None   
3            None         None  None           None               None   
4            None         None  None           None               None   
5            None         None  None           None               None   
6            None         None  None           None               None   
7            None         None  None           None               None   
8            None         None  None           None               None   
9            None         None  None           None               None   
10           None         None  None           None               None   
11           None         None  None           None               None   
12           None         None  None           None               None   
13           None         None  None           None               None   
14           None         None  None           None               None   

   Image Status      Reject Reason  
0      APPROVED                NaN  
1      APPROVED                NaN  
2      REJECTED  Positioning error  
3      APPROVED                NaN  
4      APPROVED                NaN  
5      APPROVED                NaN  
6      APPROVED                NaN  
7      APPROVED                NaN  
8      APPROVED                NaN  
9      APPROVED                NaN  
10     APPROVED                NaN  
11     APPROVED                NaN  
12     APPROVED                NaN  
13     APP

## Output of dataframe into .csv file

Check the output table above and confirm that the information matches the column names. 
If everything is OK, let's export the cleaned up .csv file into an output file. 

A .csv file will be created with the Asset Number and a datestamp.

In [47]:
# Add timestamp to filename
from datetime import datetime
date = datetime.now().strftime("%Y_%m_%d_%I_%S%p")

df_out.to_csv((r'C:\Users\BernardM\GitHub\JupyterNotebooks\rejectAnalysis\outputdata\\'
               +str(AssetNumber)
               +'_'
               +str(date)
               +'.csv')
               ,index = False, header = True)

print("The output file has been successfully created.")

The output file has been successfully created.


This output file can now be appended to the Reject Analysis and Dose Metric Dashboard database.

Add a unit test for ENTRIES in BODY PART that don't match the "ANATOMICAL REGION" column